In [3]:
import requests
import urllib
import re
from bs4 import BeautifulSoup
from googlesearch import search
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
import time
import pickle

In [4]:
cities_data = pd.read_csv("C:\\Users\\Crist\\AppData\\Local\\Temp\\American-Community-Survey-Project\\cal_cities_lat_long.csv")

In [5]:
cities_data.Name.size

458

In [6]:
city_list = cities_data.Name.tolist()
list_of_cities = []
for city in city_list:
    new = city.replace(' ','-')
    list_of_cities.append(new)

In [7]:
list_of_cities

['Adelanto',
 'Agoura-Hills',
 'Alameda',
 'Albany',
 'Alhambra',
 'Aliso-Viejo',
 'Alturas',
 'Amador-City',
 'American-Canyon',
 'Anaheim',
 'Anderson',
 'Angels',
 'Antioch',
 'Arcadia',
 'Arcata',
 'Arroyo-Grande',
 'Artesia',
 'Arvin',
 'Atascadero',
 'Atwater',
 'Auburn',
 'Avalon',
 'Avenal',
 'Azusa',
 'Bakersfield',
 'Baldwin-Park',
 'Banning',
 'Barstow',
 'Beaumont',
 'Bell-Gardens',
 'Bellflower',
 'Belmont',
 'Belvedere',
 'Berkeley',
 'Beverly-Hills',
 'Big-Bear-Lake',
 'Biggs',
 'Bishop',
 'Blue-Lake',
 'Blythe',
 'Bradbury',
 'Brawley',
 'Brea',
 'Brentwood',
 'Brisbane',
 'Buellton',
 'Buena-Park',
 'Burbank',
 'Burlingame',
 'Calabasas',
 'Calexico',
 'California-City',
 'Calimesa',
 'Calipatria',
 'Calistoga',
 'Camarillo',
 'Campbell',
 'Canyon-Lake',
 'Capitola',
 'Carlsbad',
 'Carmel-By-the-Sea',
 'Carpinteria',
 'Carson',
 'Cathedral-City',
 'Ceres',
 'Cerritos',
 'Chico',
 'Chino',
 'Chino-Hills',
 'Chowchilla',
 'Chula-Vista',
 'Citrus-Heights',
 'Claremont',
 

In [8]:
clean_links = []
missing_cities = []

In [ ]:
for city in list_of_cities:
    #only get links for cities not in clean_links
        backup_url = "https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html".format(city)
        query  = "'https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html'".format(city)
        query = urllib.parse.quote_plus(query) # Format into URL encoding
        number_result = 3
        
        ua = UserAgent()
        
        google_url = "https://www.google.com/search?q=" + query + "&num=" + str(number_result)
        t0 = time.time()
        response = requests.get(google_url, {"User-Agent": ua.random})
        response_delay = time.time() - t0
        time.sleep(10*response_delay)
        soup = BeautifulSoup(response.text, "html.parser")
        
        result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})
        
        links = []
        for r in result_div:
            
            try:
                link = r.find('a', href = True)

                if link != '':
                    links.append(link['href'])
            except:
                continue
        flag = True
        for i, l in enumerate(links):
            clean = re.search('\/url\?q\=(.*)\&sa',l)
            if clean is None:
                continue
            elif city in clean.group(1):
                clean_links.append(clean.group(1))
                flag = False
        if flag:
            missing_cities.append(city)
            clean_links.append(backup_url)

Not perfect so we will:
    *remove identical strings
    *remove strings that have the word county in them
    *track http error 404 if our self made url fails

Pickle the output

In [ ]:
# outfile = open('my_clean_links','wb')
# pickle.dump(clean_links,outfile)
# outfile.close()

# outfile = open('unfound_urls','wb')
# pickle.dump(missing_cities,outfile)
# outfile.close()

Load the the clean_links

In [52]:

# infile = open('my_clean_links','rb')
# clean_links = pickle.load(infile)
# infile.close()

# infile = open('unfound_urls','rb')
# unfound_urls = pickle.load(infile)
# infile.close()

In [53]:
clean_links = pd.Series(clean_links)

In [12]:
# #remove county results
# without_counties = pd.Series(clean_links)
# without_counties = without_counties.mask(pd.Series(clean_links).str.contains('County'))
# without_counties = without_counties.dropna()

In [54]:
#delete duplicates
city_urls = clean_links.drop_duplicates()

In [55]:
#need to make sure to reset the indexes
city_urls = city_urls.reset_index(drop=True)

Housing tab integrity check

In [56]:
city_urls

0      https://www.towncharts.com/California/Housing/...
1      https://www.towncharts.com/California/Housing/...
2      https://www.towncharts.com/California/Housing/...
3      http://www.towncharts.com/California/Housing/A...
4      http://www.towncharts.com/California/Housing/A...
5      https://www.towncharts.com/California/Housing/...
6      http://www.towncharts.com/California/Housing/A...
7      https://www.towncharts.com/California/Housing/...
8      https://www.towncharts.com/California/Housing/...
9      https://www.towncharts.com/California/Housing/...
10     https://www.towncharts.com/California/Housing/...
11     https://www.towncharts.com/California/Housing/...
12     https://www.towncharts.com/California/Housing/...
13     https://www.towncharts.com/California/Housing/...
14     https://www.towncharts.com/California/Housing/...
15     https://www.towncharts.com/California/Housing/...
16     http://www.towncharts.com/California/Housing/A...
17     https://www.towncharts.c

In [57]:
indexes_to_replace = []

In [58]:

for i,v in enumerate(city_urls):
    v = pd.Series(v)
    check = v.str.extract('\/California\/(.*?)\/',expand=False)
    if 'Housing' not in check[0]:
         print(check[0])
         indexes_to_replace.append(i)


Economy
Economy
Demographics
Economy


In [59]:
indexes_to_replace

[22, 43, 74, 77]

Consistent format with city not CDP or CDD

In [88]:
for i,v in enumerate(city_urls):
    v=pd.Series(v)
    check = re.findall('City',v[0])
    check2 = re.findall('city',v[0])
    if ( (check == []) & (check2 == [])):
        indexes_to_replace.append(i)

In [127]:
indexes_to_replace

[22, 43, 74, 77, 3, 29, 40, 51, 67, 73, 85, 90]

In [139]:
unintended_urls = pd.Series()

In [140]:
for i in indexes_to_replace:
    unintended_urls = unintended_urls.append( pd.Series([city_urls[i]]),ignore_index=True )

In [167]:
for i in unintended_urls:
    print(i)

http://www.towncharts.com/California/Economy/Avalon-city-CA-Economy-data.html
http://www.towncharts.com/California/Economy/Blythe-city-CA-Economy-data.html
https://www.towncharts.com/California/Demographics/Chico-city-CA-Demographics-data.html
https://www.towncharts.com/California/Economy/Chowchilla-city-CA-Economy-data.html
http://www.towncharts.com/California/Housing/Alameda-County-CA-Housing-data.html
http://www.towncharts.com/California/Housing/Barstow-CCD-CA-Housing-data.html
http://www.towncharts.com/California/Housing/Bishop-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Burbank-CDP-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Carpinteria-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Chico-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Coachella-Valley-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Central-Colusa-CCD-CA-Housing-data.html


In [157]:
len(unintended_urls)

12

In [158]:
appropriate_names = []
for city in list_of_cities:
    if not unintended_urls[unintended_urls.str.contains(city)].empty:
        appropriate_names.append(city)

In [161]:
appropriate_names

['Alameda',
 'Avalon',
 'Barstow',
 'Bishop',
 'Blythe',
 'Burbank',
 'Carpinteria',
 'Chico',
 'Chowchilla',
 'Coachella',
 'Colusa']

In [171]:
fix_cities = []
mycount = 0
while mycount != len(unintended_urls):
    for i in unintended_urls:
        for l in appropriate_names:
            if l in i:
                fix_cities.append(l)
                
                mycount +=1

In [106]:
# for i in indexes_to_replace:
#     v = pd.Series(city_urls[i])
#     check = v.str.extract(r'nia\/.*?\/(.*?)-')
#     print(check)

        0
0  Avalon
        0
0  Blythe
       0
0  Chico
            0
0  Chowchilla
         0
0  Alameda
         0
0  Barstow
        0
0  Bishop
         0
0  Burbank
             0
0  Carpinteria
       0
0  Chico
           0
0  Coachella
         0
0  Central


In [ ]:
# take the indexes out add backup url only 
# for i in list:
#     city_urls.str.contains('name')

In [184]:
fix_cities

['Avalon',
 'Blythe',
 'Chico',
 'Chowchilla',
 'Alameda',
 'Barstow',
 'Bishop',
 'Burbank',
 'Carpinteria',
 'Chico',
 'Coachella',
 'Colusa']

In [189]:
to_fix = pd.Series(fix_cities,index = indexes_to_replace)
to_fix

22         Avalon
43         Blythe
74          Chico
77     Chowchilla
3         Alameda
29        Barstow
40         Bishop
51        Burbank
67    Carpinteria
73          Chico
85      Coachella
90         Colusa
dtype: object

In [177]:
#once we take them out 
city_urls = city_urls.drop(labels = indexes_to_replace)

In [179]:

for city in list_of_cities:
    if city_urls[city_urls.str.contains(city)].empty:
        print(city)

Avalon
Chowchilla
Coachella


can add them where they were before (22,77,85) with our backup url
anywhere but since its just three we can

Note:city_urls indexes have not been reset


In [190]:
cities_to_fix = pd.Series(['Avalon','Chowchilla','Coachella'],index = [22,77,85])


In [191]:
cities_to_fix

22        Avalon
77    Chowchilla
85     Coachella
dtype: object

In [193]:
cop_city_urls = city_urls.copy()

In [195]:
for i,v in cities_to_fix.items():
    cop_city_urls[i] = "https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html".format(v)
    print(cop_city_urls[i])

https://www.towncharts.com/California/Housing/Avalon-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Chowchilla-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Coachella-city-CA-Housing-data.html


In [196]:
len(cop_city_urls)

458

In [197]:
#double check
# #delete duplicates if accidentally created any
# searchurls = cop_city_urls.drop_duplicates()
# len(searchurls)

458

Now we actually scrape the desired attributes

In [198]:
results = []
not_found = []

In [104]:
def get_text_section(target_section,mysoup):
    """
    target_section:
    returns: 
    """
    found = mysoup.find_all('div',attrs = {'class':target_section})
    validsection = found[0]
    text_section = validsection.find_all(text=True)
    return text_section

In [105]:
def searchpattern(pattern,resultset):
    '''
    pattern:regex pattern
    resultset: A beautiful soup result set
    output housing variable of interest type string
    '''
    result = [i for i in resultset if re.search(pattern,i)]
    if result == []:
        result  = None
    elif result !=[]:
        result = (re.findall(pattern,result[0]))[0]
    return result

In [106]:
def refinestring(desiredreplacement,yourstring):
    """
    yourstring: a string which you extract only digits and decimals
    desiredreplacement: a dictonary definining your replacement values
    
    returns: your string with the desired replacements you request 
    """
    if yourstring  == None:
        return None
    elif yourstring != None:
        rep = dict((re.escape(k),v) for k,v in desiredreplacement.items())
        pattern =re.compile("|".join(rep.keys()))

        return float(pattern.sub(lambda m:rep[re.escape(m.group(0))],yourstring))

In [107]:
for aurl in searchurls:
    ua = UserAgent()
    t0 = time.time()
    response = requests.get(aurl,{"User-Agent": ua.random})
    
    if response.status_code != 200:
        not_found.append(aurl)
        continue
        
    response_delay = time.time() - t0
    time.sleep(10*response_delay)
    soup = BeautifulSoup(response.text,'html.parser')
    
    
    housing_text_section1 = get_text_section('section1',soup)
    housing_text_section2 = get_text_section('section2',soup)
    housing_text_section3 = get_text_section('section4',soup)
    
    #gathering variables
    
    city_name = re.search('(.*)\sCalifornia',housing_text_section1[0]).group(1)

    #for variables need to remove everything but numbers
    #figure1 
    housing_units = searchpattern('it\shas\s(\d.*?\d)\shousing\sunits\s',housing_text_section1)
    housing_units = refinestring({"$":"",",":"","%":""},housing_units)


    #figure2 
    housing_density = searchpattern('it\shas\s(\d.*?\d)\shousing\sdensity\s',housing_text_section1)
    housing_density = refinestring({"$":"",",":"","%":""},housing_density)

    #figure4 
    change_hunits = searchpattern('it\shas\s(.*?%)\schange\sin\s',housing_text_section1)
    change_hunits = refinestring({"$":"",",":"","%":""},change_hunits)

    #figure5
    percent_of_rent_to_total = searchpattern('it\shas\s(\d.*?%)\sRenter\sPercent\s',housing_text_section1)
    percent_of_rent_to_total = refinestring({"$":"",",":"","%":""},percent_of_rent_to_total)


    #figure 6
    owned_homes = searchpattern('it\shas\s(\d.*?%)\sOwner\sPercent\s',housing_text_section1)
    owned_homes = refinestring({"$":"",",":"","%":""},owned_homes)

    #figure 7 median home value (unique character  = $-numbers-commas)
    med_homeval = searchpattern('it\shas\s(.*?\d)\smedian\shome\svalue\s',housing_text_section2)
    med_homeval = refinestring({"$":"",",":"","%":""},med_homeval)

    #figure 8 median rental rate
    med_rental_rate = searchpattern('it\shas\s(.*?\d)\srental\srates\s',housing_text_section2)
    med_rental_rate = refinestring({"$":"",",":"","%":""},med_rental_rate)

    #figure 10 median owner costs
    med_owner_cost = searchpattern("it\shas\s(.*?\d)\smedian\sownership\scost\s",housing_text_section2)
    med_owner_cost = refinestring({"$":"",",":"","%":""},med_owner_cost)

    #figure 11
    med_own_cost_aspercentof_income = searchpattern('median\sowner\scost\sas\sa\spercent\sof\stotal\shousehold\sincome\sand\sit\shas\s(\d.*?%)',housing_text_section2)
    med_own_cost_aspercentof_income = refinestring({"$":"",",":"","%":""},med_own_cost_aspercentof_income)


    #figure 12 median home value as a percent of a median workers earnings
    med_hval_aspercentof_medearn = searchpattern("it\shas\s(\d.*?%)\smedian\shome\svalue\sas\s",housing_text_section2)
    med_hval_aspercentof_medearn = refinestring({"$":"",",":"","%":""},med_hval_aspercentof_medearn)


    #figure14
    med_hcost_ownmortg = searchpattern('depicts.*\$(.*?\d)\smedian',housing_text_section2)
    med_hcost_ownmortg = refinestring({"$":"",",":"","%":""},med_hcost_ownmortg)


    #figure 15
    med_hcost_own_wo_mortg = searchpattern('are.*\$(.*?\d)\smedian\shousing\s',housing_text_section2)
    med_hcost_own_wo_mortg = refinestring({"$":"",",":"","%":""},med_hcost_own_wo_mortg)

    #figure 17
    hcost_aspercentof_hincome_ownmortg= refinestring({"$":"",",":"","%":""},searchpattern('with\smortgages.*\s(\d.*?%)\scost',housing_text_section2))



    #figure18
    hcost_as_perc_of_hincome_womortg = refinestring({"$":"",",":"","%":""},searchpattern('This\sanalysis.*\s(\d.*?%)\scost',housing_text_section2))


    #figure 21 **
    med_real_estate_taxes = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\smedian\sreal\sestate\s',housing_text_section2))
    #figure 24 biggie see below

    #figure 28 format 3.0
    family_members_per_hunit = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\sfamily\smembers\sper\s',housing_text_section3))


    #figure 29 4.5
    median_num_ofrooms = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\saverage\snumber\sof\s',housing_text_section3))


    #figure 30
    median_year_house_built = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\syear\sbuilt\swhich\s',housing_text_section3))


    #figure 36
    household_size_of_howners = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\shomeowner\shousehold\ssize\s',housing_text_section3))


    #figure 37
    household_size_for_renters = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\srenter\shousehold\ssize\s',housing_text_section3))
    #figure 40
    med_year_moved_in_for_owners = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\smedian\syear\sthat\sa\shome',housing_text_section3))
    #figure41
    med_year_renter_moved_in = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\smedian\syear\sthat\sa\srenter',housing_text_section3))
    
    #scraping javascript table
    
    javascript = soup.find_all('script',attrs = {'type':'text/javascript'})
    need = javascript[4]
    needtext = str(need.find_all(text=True))
    fig24 = re.search('data24\s\=\sgoogle\.visualization\.arrayToDataTable\((.*?)\);',needtext).group(1)
    list_strs = re.findall(r'\[(.*?)\]',fig24)
    #gathering the variables in the table
    check1 = True
    check2 = True
    check3 = True
    for i,v in enumerate(list_strs):
        if '$1,000 to $1,499' in v:
            check1 = False
            val1 = re.findall('v\:(.*?\d),',v)
            studio_1000_1499 = float(val1[0])
            onebed_1000_1499 = float(val1[1])
            twobed_1000_1499 = float(val1[2])
            threebed_1000_1499 = float(val1[3])
        elif '$1,500 or more' in v:
            check2 = False
            val2 = re.findall('v\:(.*?\d),',v)
            studio_1500_more = float(val2[0])
            onebed_1500_more = float(val2[1])
            twobed_1500_more = float(val2[2])
            threebed_1500_more = float(val2[3])
        elif '$750 to $999' in v:
            check3 = False
            val3 = re.findall('v\:(.*?\d),',v)
            studio_750_999 = float(val3[0])
            onebed_750_999 = float(val3[1])
            twobed_750_999 = float(val3[2])
            threebed_750_999 = float(val3[3])
    
    if check1:
        studio_1000_1499 = None
        onebed_1000_1499 = None
        twobed_1000_1499 = None
        threebed_1000_1499 = None

    elif check2:
        studio_1500_more = None
        onebed_1500_more = None
        twobed_1500_more = None
        threebed_1500_more = None

    elif check3:
        studio_750_999 = None
        onebed_750_999 = None
        twobed_750_999 = None
        threebed_750_999 = None
    
    results.append((city_name,housing_units,housing_density,change_hunits,percent_of_rent_to_total,owned_homes,
           med_homeval,med_rental_rate,med_owner_cost,med_own_cost_aspercentof_income,med_hval_aspercentof_medearn,
          med_hcost_ownmortg,med_hcost_own_wo_mortg,hcost_aspercentof_hincome_ownmortg,hcost_as_perc_of_hincome_womortg,
          med_real_estate_taxes,family_members_per_hunit,median_num_ofrooms,median_year_house_built,household_size_of_howners,household_size_for_renters,
          med_year_moved_in_for_owners,med_year_renter_moved_in,studio_1000_1499,studio_1500_more,studio_750_999,onebed_1000_1499,onebed_1500_more,onebed_750_999,
          twobed_1000_1499,twobed_1500_more,twobed_750_999,threebed_1000_1499,threebed_1500_more,threebed_750_999))
    

AttributeError: 'NoneType' object has no attribute 'group'

In [93]:
df = pd.DataFrame(results,columns =['city_name','housing_units','housing_density','change_hunits','percent_of_rent_to_total','owned_homes',
                'med_homeval','med_rental_rate','med_owner_cost','med_own_cost_aspercentof_income','med_hval_aspercentof_medearn',
                'med_hcost_ownmortg','med_hcost_own_wo_mortg','hcost_aspercentof_hincome_ownmortg','hcost_as_perc_of_hincome_womortg',
                'med_real_estate_taxes','family_members_per_hunit','median_num_ofrooms','median_year_house_built','household_size_of_howners','household_size_for_renters',
                'med_year_moved_in_for_owners','med_year_renter_moved_in','studio_1000_1499','studio_1500_more','studio_750_999','onebed_1000_1499','onebed_1500_more',
                 'onebed_750_999','twobed_1000_1499','twobed_1500_more','twobed_750_999','threebed_1000_1499','threebed_1500_more','threebed_750_999'] )

In [108]:
results

[('Adelanto',
  8751.0,
  156.2,
  -3.7,
  50.7,
  49.3,
  151600.0,
  1059.0,
  1093.0,
  24.0,
  427.0,
  1260.0,
  406.0,
  28.0,
  15.0,
  1766.0,
  4.4,
  5.5,
  1997.0,
  3.7,
  4.1,
  2007.0,
  2013.0,
  0.0,
  0.0,
  0.379,
  0.003,
  0.032,
  0.096,
  0.131,
  0.032,
  0.508,
  0.583,
  0.137,
  0.181),
 ('Agoura Hills',
  7674.0,
  984.7,
  1.2,
  25.6,
  74.4,
  745000.0,
  2261.0,
  2488.0,
  23.0,
  928.0,
  2943.0,
  844.0,
  26.0,
  10.0,
  5345.0,
  3.3,
  6.5,
  1980.0,
  2.8,
  2.8,
  2001.0,
  2012.0,
  0.0,
  1.0,
  0.0,
  0.105,
  0.895,
  0.0,
  0.064,
  0.899,
  0.038,
  0.0,
  0.962,
  0.0),
 ('Alameda city-CA',
  32414.0,
  3104.1,
  0.2,
  53.0,
  47.0,
  729100.0,
  1607.0,
  2259.0,
  21.0,
  1006.0,
  2833.0,
  679.0,
  24.0,
  10.0,
  6036.0,
  3.2,
  4.8,
  1962.0,
  2.7,
  2.4,
  2000.0,
  2011.0,
  0.397,
  0.312,
  0.142,
  0.461,
  0.394,
  0.058,
  0.261,
  0.633,
  0.019,
  0.119,
  0.667,
  0.036),
 ('Albany',
  7724.0,
  4319.4,
  -2.1,
  52.4,
  

In [95]:
df

,city_name,housing_units,housing_density,change_hunits,percent_of_rent_to_total,owned_homes,med_homeval,med_rental_rate,med_owner_cost,med_own_cost_aspercentof_income,...,studio_750_999,onebed_1000_1499,onebed_1500_more,onebed_750_999,twobed_1000_1499,twobed_1500_more,twobed_750_999,threebed_1000_1499,threebed_1500_more,threebed_750_999


In [ ]:
# thelist = (city_name,housing_units,housing_density,change_hunits,percent_of_rent_to_total,owned_homes,
#            med_homeval,med_rental_rate,med_owner_cost,med_own_cost_aspercentof_income,med_hval_aspercentof_medearn,
#           med_hcost_ownmortg,med_hcost_own_wo_mortg,hcost_aspercentof_hincome_ownmortg,hcost_as_perc_of_hincome_womortg,
#           med_real_estate_taxes,family_members_per_hunit,median_num_ofrooms,median_year_house_built,household_size_of_howners,household_size_for_renters,
#           med_year_moved_in_for_owners,med_year_renter_moved_in,studio_1000_1499,studio_1500_more,studio_750_999,onebed_1000_1499,onebed_1500_more,onebed_750_999,
#           twobed_1000_1499,twobed_1500_more,twobed_750_999,threebed_1000_1499,threebed_1500_more,threebed_750_999)

# thestringlist = ['city_name','housing_units','housing_density','change_hunits','percent_of_rent_to_total','owned_homes',
#                 'med_homeval','med_rental_rate','med_owner_cost','med_own_cost_aspercentof_income','med_hval_aspercentof_medearn',
#                 'med_hcost_ownmortg','med_hcost_own_wo_mortg','hcost_aspercentof_hincome_ownmortg','hcost_as_perc_of_hincome_womortg',
#                 'med_real_estate_taxes','family_members_per_hunit','median_num_ofrooms','median_year_house_built','household_size_of_howners','household_size_for_renters',
#                 'med_year_moved_in_for_owners','med_year_renter_moved_in','studio_1000_1499','studio_1500_more','studio_750_999','onebed_1000_1499','onebed_1500_more',
#                  'onebed_750_999','twobed_1000_1499','twobed_1500_more','twobed_750_999','threebed_1000_1499','threebed_1500_more','threebed_750_999']